In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("data/free_throws.csv")

In [3]:
data.head()

,end_result,game,game_id,period,play,player,playoffs,score,season,shot_made,time
0,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,0 - 1,2006 - 2007,1,11:45
1,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 2 of 2,Andrew Bynum,regular,0 - 2,2006 - 2007,1,11:45
2,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,1,7:26
3,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum misses free throw 2 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,0,7:26
4,106 - 114,PHX - LAL,261031013.0,1.0,Shawn Marion makes free throw 1 of 1,Shawn Marion,regular,21 - 12,2006 - 2007,1,7:18


In [2]:
from statsmodels.stats.weightstats import ztest

new_df = {
    "player": [],
    "regular_mean": [],
    "playoff_mean": [],
    "p_value": []
}

for player, group in data.groupby("player"):
    regular_shots = group[group["playoffs"] == "regular"]["shot_made"].values
    playoff_shots = group[group["playoffs"] == "playoffs"]["shot_made"].values
    
    if len(regular_shots) < 30 or len(playoff_shots) < 30:
        continue
        
    statistic, p_value = ztest(regular_shots, playoff_shots)

    new_df["player"].append(player)
    new_df["regular_mean"].append(np.mean(regular_shots))
    new_df["playoff_mean"].append(np.mean(playoff_shots))
    new_df["p_value"].append(p_value)
    
new_df = pd.DataFrame(new_df)

In [4]:
new_df.sample(10)

,player,regular_mean,playoff_mean,p_value
170,Marcus Smart,0.721649,0.694444,0.732758
251,Tony Parker,0.780479,0.734426,0.013941
67,Delonte West,0.826645,0.849057,0.570664
24,Ben Gordon,0.866342,0.899160,0.304904
186,Mike Scott,0.783537,0.800000,0.834063
177,Matt Harpring,0.747405,0.761905,0.774824
201,Paul George,0.837629,0.797753,0.070627
203,Paul Pierce,0.827090,0.835347,0.604012
183,Mike Bibby,0.800525,0.746479,0.280064
187,Mikki Moore,0.715164,0.800000,0.280189


In [5]:
new_df[new_df.p_value<=0.05].shape

(22, 4)

In [9]:
from statsmodels.stats.multitest import multipletests

multipletests(new_df.p_value, alpha=0.05, method='bonferroni')[0].sum()

1

In [11]:
multipletests(new_df.p_value, alpha=0.05, method='holm')[0].sum()

1